In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings 
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Data Cleaning**
.
.
.
.
.

In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')

train.describe(include = "all")

In [ ]:
#Análise inicial

#a. Quais as colunas do banco de dados de teste e de treino
print("As colunas do banco 'Train' são:\n")
print(train.columns)

print("\nAs colunas do banco 'Test' são:\n")
print(test.columns)

#b. Qual o tipo de dado de cada coluna nos dataframes de teste e de treino
print("\n")
train.info()
test.info()

#c. Qual a quantidade de valores nulos (NaN) em cada feature
print("\n")
print(pd.isnull(train).sum())
print("\n")
print(pd.isnull(test).sum())

#d. Realizar um cópia do banco de dados de teste e de treino para que se possa
# fazer a manipulação sem perder informações
trainCopy = train.copy()

In [ ]:
#Para lidar com valores nulos, podemos preencher estes valores de alguma forma ou
# descartar a informação. Neste item utilizaremos algumas estratégias para tal.

#a. A feature “Cabin” contém muitos valores nulos e, assim como “Ticket”, não
# fornece informações relevantes ao modelo à primeira vista. Então pode-se
# retirá-las.

# (laço de repetição para englobar os dois bancos de dados ao mesmo tempo)
for data in [trainCopy, test]:
    data = data.drop(['Cabin','Ticket'], axis = 1, inplace = True)

#b. A coluna “Age” também contém muitos valores nulos, mas dessa vez iremos
# preenchê-los. Utilizar a mediana dos valores de idade para isso, é uma boa
# estratégia inicial.

#c. A feature “Fare” contém poucos valores nulos, e pode ser útil para análises
# futuras. Pode-se preenchê-la com a mesma estratégia que “Age”, já que
# ambas são contínuas.

#d. Em “Embarked” também tem-se pouca ocorrência de valores nulos, e como é
# uma feature categórica, preencher com a moda parece menos impactante na
# construção do modelo.
for data in [trainCopy, test]:
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['Fare'].fillna(data['Fare'].median(), inplace = True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)


print(trainCopy.isnull().sum())
print("\n")
print(test.isnull().sum())

In [ ]:
#Para as Features contínuas será útil a criação de grupos para facilitar a análise

#a. Criar Feature que separe a Feature “Age” em 5 intervalos de mesma extensão.
for data in [trainCopy,test]:
    data['Age_group'] = pd.cut(data['Age'].astype(int), 5)
#b. Criar Feature que separe “Fare” em 6 intervalos que contenham o mesmo número de dados (Não precisam ter a mesma extensão).
    data['Fare_group'] = pd.qcut(data['Fare'], 6)
    
display(trainCopy)
display(test)

**Aperfeiçoamento da Modelagem**
.
.
.
.
.


In [ ]:
#Será necessário realizar o encoding das variáveis categóricas, no momento, 3
# tratégias que serão utilizadas são o one-hot, label encoding e ordinal encoding.
# para fazer isso será necessário relembrar quais são as features categóricas, definir a
# estratégia que será utilizada, criar as features codificadas e retirar as categóricas

trainCopy.info()
print("\nColunas numéricas:")
print(trainCopy.select_dtypes(include = ("int64","float64")))
print("\nColunas categóricas:")
print(trainCopy.select_dtypes(include = ("object","category")))

In [ ]:
#Para features sem ordem definida, one-hot encoding pode ser a melhor opção.
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse = False)

#Criar uma função que realize o one-hot encode e, como saída, retorna um
# novo dataframe com as colunas que resultam da codificação, devidamente
# nomeadas, ao invés das features categóricas:

def OH_encoder(data,cols):
    data_encoded = data.copy()
    for col in cols:      
        #Criando colunas para One-hot-encode
        OH_cols = pd.DataFrame(onehot.fit_transform(data[[col]]),dtype = 'int')
        
        #Nomeando as colunas
        OH_cols.columns = onehot.get_feature_names([col])
        
        #Adicionando as novas colunas codificadas ao dataframe
        data_encoded = data_encoded.drop([col], axis = 1)
        data_encoded = pd.concat([data_encoded,OH_cols], axis = 1)
    return data_encoded

#Realizando o encoding dos dataframes
train_encoded = OH_encoder(trainCopy,["Sex","Embarked"])
test_encoded = OH_encoder(test,["Sex","Embarked"])


In [ ]:
#Para as features que possuem ordem, o label encoder é mais indicado
# como primeira abordagem.
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

#Age_group e Fare_group possuem uma sequência, ou seja, possuem ordem

#Realizando Label encoding
for data in [train_encoded,test_encoded]:
    data["Age_group_code"] = label.fit_transform(data["Age_group"])
    data["Fare_group_code"] = label.fit_transform(data["Fare_group"])
    data = data.drop(["Age_group","Fare_group","Name"], axis = 1, inplace = True)

display(train_encoded)

In [ ]:
# Agora será realizado um novo treinamento e avaliação de modelo, os passos serão os
# mesmo realizados no item 6.1 g), mas agora com o banco de dados que é resultado
# do encoding das variáveis categóricas. Após essa avaliação, será possível notar a
# evolução da precisão do modelo produzido utilizando essas novas técnicas.


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

randomforest_model1 = RandomForestClassifier(random_state = 0)

X_model1 = train_encoded.drop('Survived', axis = 1)
y_model1 = train_encoded['Survived']

X_train,X_valid,y_train,y_valid = train_test_split(X_model1,y_model1,test_size = 0.2,random_state = 0)

randomforest_model1.fit(X_train,y_train)
preds = randomforest_model1.predict(X_valid)

accuracy = round(accuracy_score(preds,y_valid)*100,2)
print(accuracy)

In [ ]:
#Deverá ser criada uma função que produz e avalia pipelines que utilizam diversas
# estratégias de imputing de valores nulos e encoding de variáveis categóricas.

#a. Colocar de volta os valores nulos das colunas “Age” e “Embarked” do
# dataframe de treino, utilizando as colunas do banco de dados original.

trainCopy_nulo = trainCopy.copy()
trainCopy_nulo = trainCopy_nulo.drop(["Age","Embarked"], axis = 1)
trainCopy_nulo = pd.concat([trainCopy_nulo,train[["Age", "Embarked"]]], axis = 1)
print(trainCopy_nulo.isnull().sum())

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#Criar uma função que produza e avalie pipelines que utilizam estratégias indicadas nos argumentos

# data: banco de dados para construção do modelo
# encoder: estratégia de encoding - one-hot ou label encoder
# model: algortimo para produzir o modelo, inicialmente random forest classifier
# numerical_imputer: estratégia para substituir valores nulos numéricos - mean, median
# categorical_imputer: estratégia para substituir valores nulos categóricos - most frequent
def pipeline(data, encoder, model, numerical_imputer = SimpleImputer(),
                   categorical_imputer = SimpleImputer(strategy = "most_frequent")):
    
    #Definir as features e o target que serão usados para treinar o modelo e separar o banco de dados utilizando o train-test split.
    X_pipe = data.drop("Survived", axis = 1)
    y_pipe = data["Survived"]

    X_train_pipe,X_valid_pipe,y_train_pipe,y_valid_pipe = train_test_split(X_pipe,y_pipe,
                                                                           test_size = 0.2,
                                                                           random_state = 0)

    #Criando os passos do preprocessor, para features numéricas e categóricas
    num_transf = numerical_imputer
    cat_transf = Pipeline(steps = [("imputer", categorical_imputer),("encoder",encoder)])
    preprocessor = ColumnTransformer(transformers = [("num",num_transf,["Age"]),
                                                ("cat",cat_transf,["Age_group","Fare_group","Sex","Embarked"])])
    
    #Produzir pipeline utilizando o preprocessor previamente construído e o modelo do argumento “model”.
    #Realizar o treinamento da pipeline, gerar predições e avaliar utilizando accuracy score.
    pipe = Pipeline(steps = [("preprocessor", preprocessor),("model", model)])
    pipe.fit(X_train_pipe,y_train_pipe)
    preds_pipe = pipe.predict(X_valid_pipe)

    #Avaliando as predições
    accuracy = round(accuracy_score(preds_pipe,y_valid_pipe)*100,2)
    
    #A função deve retornar o resultado da avaliação em porcentagem.
    return accuracy, preds_pipe

In [ ]:
#Imputers
#Definir as 4 seguintes estratégias para imputing
mean_imputer = SimpleImputer()
mostfreq_imputer = SimpleImputer(strategy = "most_frequent")
median_imputer = SimpleImputer(strategy = "median")
zero_imputer = SimpleImputer(strategy = "constant", fill_value = 0)
imputers = [mean_imputer, mostfreq_imputer, median_imputer, zero_imputer]

#Encoders
#Definir as 2 estratégias para encoding
from sklearn.preprocessing import OrdinalEncoder
onehot_encoder = OneHotEncoder(sparse = False)
ordinal_encoder = OrdinalEncoder()
encoders = [onehot_encoder, ordinal_encoder]

#model
randomforest = RandomForestClassifier(random_state = 0)

In [ ]:
#A partir da função de criação de pipelines e das listas de encoders e imputers, criar
# um loop que produza todas as combinações possíveis dessas estratégias, e mostre o
# resultado da avaliação de precisão de cada pipeline produzida no loop

for encoder in encoders:
    for imputer in imputers:
        accuracy,_ = pipeline(trainCopy_nulo, encoder, randomforest, numerical_imputer = imputer)
        print("pipe_"+str(encoder)+'_'+str(imputer)+" = "+str(accuracy))
        
#precisão inferior àquela encontrada anteriormente. Estratégia de avaliação separada inferior à estratégia mesclada

**Cross validation e Gradient boosting**
.
.
.
.
.

In [ ]:
#O cross validation é uma maneira melhor para avaliar os modelos com os quais
# estamos lidando. Portanto, o modelo que melhor se saiu nos teste até agora, deverá
# ser avaliado utilizando este método. Como resultado será utilizada a média entre as
# avaliações

from sklearn.model_selection import cross_val_score

#Criando função para gerar avaliação
def cross_validation(model, X, y):
    scores = cross_val_score(model, X, y, cv = 5, scoring = "accuracy")
    return round(scores.mean()*100,2)

#Gerando avaliação do modelo mesclado 
accuracy_crossval = cross_validation(randomforest_model1, X_model1, y_model1)
print(accuracy_crossval)

#Assim como o Random Forest, o XGBoost também tem seu equivalente como
# classifier. O Gradient Boosting Classifier será utilizado agora para gerar um novo
# modelo, a partir do dataframe que foi manipulado com as melhores estratégias até agora

from sklearn.ensemble import GradientBoostingClassifier

#Criando Função Gradient Boosting classifier para gerar e avaliar um modelo 
def make_gbc(X,y):
    gbclass = GradientBoostingClassifier(random_state = 0, n_iter_no_change = 100) 
    gbclass.fit(X,y)
    score = cross_validation(gbclass, X, y)
    return score, gbclass

X_model2 = X_model1
y_model2 = y_model1

score,gbclass = make_gbc(X_model2,y_model2)
print(score)

**Feature Engineering**
.
.
.
.
.

In [ ]:
#Os objetivos dessa área são, selecionar as features do banco de dados que sejam
# menos relacionadas possível entre si, para evitar a redundância, e o mais correlacionadas
# possível com o target, para que, a partir da variação dessas features, o modelo consiga
# identificar a variação do target, a fim de predizê-lo.

#O primeiro passo é a avaliação da importância de cada feature que já está no banco
# de dados de teste, para isso será criada uma função que produza um gráfico indicado
# o Mutual Information score de cada feature existente no banco de dados d treino, em
# relação ao target

from sklearn.feature_selection import mutual_info_classif

#a. Determinar quais são as features discretas
discrete_features = train_encoded.drop(['Age','Fare'], axis = 1)

#b. Criar a função que produza o gráfico de MI scores
def make_MI (X, y, discrete_features):
    
    #Produzindo MI scores
    mi_scores = mutual_info_classif(X, y)
    mi_scores = pd.Series(mi_scores, index = X.columns)
    mi_scores = mi_scores.sort_values(ascending = False)
    
    #Prduzindo gráfico
    sns.set_style('whitegrid')
    scores = mi_scores.sort_values(ascending = True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    
    return mi_scores

X = train_encoded.drop(['Survived'],axis = 1)
y = train_encoded['Survived']

make_MI(X, y, discrete_features)


In [ ]:
#Após a observação do gráfico gerado é possível concluir que as features que as
# features com maior influência no target são “Fare”, “Sex” e “Age”, por este motivo, elas
# serão analisadas mais a fundo por meio da utilização de gráficos. Para facilitar a
# visualização dos dados, é útil que seja usado um banco de dados que ainda contenha
# as features categóricas originais


#a. Construir dois gráficos do tipo sns.histplot para a análise de “Fare”.
fig, axs = plt.subplots(1,2, figsize = (18,7))

#Criando gráficos da distribuição de "Fare"
axs[0].set_title("Distribuição das Taxas")
sns.histplot(x = "Fare", kde = True, data = trainCopy, ax = axs[0]);
axs[1].set_title("Relação Entre Taxas e Sobreviventes")
sns.histplot(x = "Fare",hue = "Survived", kde = True, data = trainCopy, ax = axs[1]);


#Agora serão criados 4 gráficos também utilizando histogramas com seaborn.
# Tais gráficos relacionam a Feature “Age” com “Sex”. Para facilitar a
# visualização é indicada a utilização de plt.subplots, para a visualização de mais
# de um gráfico por output.

fig, axs = plt.subplots(2,2, figsize = (18,7), sharex = True, sharey = True)

axs[0,0].set_title('Sexo em relação à idade')
sns.histplot(x = 'Age',hue = 'Sex', kde = True, data = trainCopy, ax = axs[0,0], palette = "YlOrRd");

axs[0,1].set_title('Distribuição de sobreviventes por idade, comparando os sexos')
sns.histplot(data = trainCopy.loc[trainCopy['Survived']==1], x = 'Age', hue = 'Sex', kde = True, ax = axs[0,1], palette = "YlOrRd_r");

axs[1,0].set_title('Distribuição de sobrevivência de homens')
sns.histplot(data = trainCopy.loc[trainCopy['Sex']=='male'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,0]);

axs[1,1].set_title('Distribuição de sobrevivência de mulheres')
sns.histplot(data = trainCopy.loc[trainCopy['Sex']=='female'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,1]);

**Feature Construction**
.
.
.
.
.

In [ ]:
#Este passo do feature engineering vai para o lado oposto em relação ao feature
# selection, já que aumenta o número de features. Em uma interpretação espacial, podemos
# dizer que cada feature do banco de dados é uma dimensão, feature construction aumenta
# este número de eixos do espaço do banco de dados, enquanto feature selection diminui.

#a. A partir de “Name” criar a feature “Title” para separar os valores,
# primeiramente, pela vírgula, e depois pelo ponto. Para isso deve-se utilizar o
# método str.split, lembre-se de criar uma nova coluna para armazenar estas
# informações, após isso, pode-se excluir a feature “Names”

# Adicionando "Name" à train_encoded
train_encoded = pd.concat([train_encoded,trainCopy["Name"]], axis = 1)

#Utilizando o método string split do pandas para retirar informações de texto 
for data in [train_encoded,test]:
    data["Title"] = data["Name"].str.split(',',expand = True)[1].str.split('.',expand = True)[0]
    data.drop("Name", axis = 1, inplace = True)

#Demonstrando quais os valores da coluna
print(train_encoded["Title"].value_counts())


#Criar novas Features para resumir informações de outras já existentes, uma para
# indicar o número total de familiares de cada passageiro, sejam eles primos, filhos ou
# acompanhantes, e outra para informar se o passageiro está sozinho ou não.

#a. Criar feature que indique o tamanho da família, somando “SibSp” e “Parch”
for data in [train_encoded,test]:
    data["Família"] = data["SibSp"] + data["Parch"]
    
#b. Criar feature booleana indicando se o passageiro está sozinho, sendo,
# verdadeiro caso não tenha nenhum familiar a bordo, e falso caso tenha.
    data["Sozinho"] = 1
    for i in range(len(data.index)):
        if data.loc[i,"Família"] != 0:
            data.loc[i,"Sozinho"] = 0

train_encoded.head()


**Feature Extraction**
.
.
.
.
.

In [ ]:
#1. No banco de dados em que estamos trabalhando existem duas features com valores
# contínuos, são "Fare'' e “Age”, portanto, a partir da interação entre elas pode-se criar
# grupos utilizando K-means como forma de clustering


#Visualizando a interação entre "Fare" e "Age"
sns.scatterplot(x="Fare",y="Age",data = trainCopy);

from sklearn.cluster import KMeans

#outliers - dados extremamentes fora do padrão
#Criando dataframe com "Age" e "Fare" sem outliers
fare_age_group = trainCopy.loc[trainCopy["Fare"] < 500,["Fare","Age"]]

#Criando feature a partir da interação entre "Age" e "Fare"
kmeans = KMeans(n_clusters = 6, random_state = 0)
fare_age_group["Fare_age_group"] = kmeans.fit_predict(fare_age_group)
fare_age_group["Fare_age_group"] = fare_age_group["Fare_age_group"].astype("int")

fare_age_group


In [ ]:
#c. Agora será útil entender quais as características de cada grupo criado pelo
# algoritmo, para isso serão utilizados dois gráficos, um para identificar os
# grupos na distribuição “Age” x “Fare”, outro para avaliá-los em relação à
# chance de sobrevivência.

#Adicionando feature criada ao banco de dados de treino
trainCopy['Fare_age_group'] = fare_age_group['Fare_age_group']

#Criando visualização para entendimento da nova feature
fig, axs = plt.subplots(1,2, figsize = (20,7))
sns.barplot(x = 'Fare_age_group', y = 'Survived', data = trainCopy, ax = axs[1], palette = "rainbow_r")
sns.scatterplot(x='Fare',y='Age', hue = 'Fare_age_group', data = trainCopy, ax = axs[0],palette = "rainbow_r" );

In [ ]:
#Ao analisarmos a nova feature, é fácil notar que existem valores nulos, estes valores
# correspondem aos outliers que foram retirados para a construção dos grupos. Para
# substituir os NaN, deve-se identificar quem são esses outliers e designar,
# manualmente, seu grupo

#Descobrindo outliers
trainCopy.loc[trainCopy["Fare_age_group"].isnull() == True,:]

#Selecionando um grupo para os Outliers
trainCopy.loc[trainCopy["Fare_age_group"].isnull() == True,["Fare_age_group"]] = 1
trainCopy.loc[[258,679,737]]

#Adicionando a nova feature aos banco de dado train_encoded
train_encoded["Fare_age_group"] = trainCopy["Fare_age_group"]
train_encoded

In [ ]:
#As features originais não representam a mesma grandeza, “Age” conta a idade em
# anos e “Fare” a taxa de embarque em dólares, portanto, para que seja possível utilizar
# o PCA, deve-se primeiramente realizar os scaling das variáveis. para esta etapa será
# utilizado o MinMax Scaler do scikit learn


from scipy import stats
from sklearn.preprocessing import MinMaxScaler

#Criando scaller 
scaller = MinMaxScaler(copy = False)

#Criando dataframe para gravar as features após realizar scalling
stand_data = pd.DataFrame()

#Loop para a realização do scalling
for column in ["Fare","Age"]:
    stand_data[[column]] = trainCopy[[column]]
    stand_data[[column]] = scaller.fit_transform(stand_data[[column]])
    
#Gerando gráficos para mostrar a distribuição das features após o scalling 
fig, axs = plt.subplots(1,2, figsize = (18,7))

sns.histplot(x = "Age", kde = True, data = stand_data, ax = axs[0]);
sns.histplot(x = "Fare", kde = True, data = stand_data, ax = axs[1]);

In [ ]:
#Agora já é possível realizar o PCA, contudo, não há garantias que esta estratégia trará
# novas informações úteis para o modelo, portanto, é necessário, após a criação dos
# novos eixos, analisar criticamente se vale a pena introduzi-los ao banco de dados.

from sklearn.decomposition import PCA

#Realizando o PCA de "Fare" e "Age" 
pca = PCA()
pca_data = pca.fit_transform(stand_data)

#Criando um dataframe para armazenar as features criadas a partir do PCA
columns = ['PC1','PC2']
pca_data = pd.DataFrame(pca_data, columns = columns)

#Criando dataframe para identificar como os eixos foram afetados pelo PCA
loadings = pd.DataFrame(pca.components_.T, columns = columns, index = ["Fare","Age"])
print(loadings)
print("\n")
print(pca_data)

(sem interação entre os eixos: PC1 ~ Age e PC2 ~ Fare)

In [ ]:
#Gerando MI scores das features PC1 e PC2
make_MI(pca_data, trainCopy["Survived"], discrete_features = False)

Score MI muito baixos, não se mostrando válido adiconar ao banco de testes

**Target Encoding**
.
.
.
.
.

In [ ]:
#Até agora, quase todas as features categóricas já foram codificadas, apenas uma
# ainda resta. No item 9.2 foi criada a feature “Title”, a partir de “Name”, essa coluna contém
# diversos valores únicos, de forma que é promissora a utilização do target encoding para
# codificá-la. Esta estratégia utiliza a relação da feature categórica com o target para substituir
# seus valores, ou seja, é uma estratégia considerada de supervised learning. Para o target
# encoding será utilizado o algoritmo MEstimateEncoder

# Deve-se então realizar o encoding da feature “Title”, após isso será interessante
# analisar a distribuição dos valores resultantes e compará-los com o target

#a. Encoding da feature “Title”
from category_encoders import MEstimateEncoder

encoder = MEstimateEncoder(cols = ["Title"], m = 5)

title = encoder.fit_transform(train_encoded["Title"], train_encoded["Survived"])
print(title)

#b. Plot da distribuição dos valores codificados
fig, ax = plt.subplots(figsize = (20,9))

sns.distplot(train_encoded["Survived"], kde=False, norm_hist=True)
sns.kdeplot(title["Title"], color='r', ax=ax)
ax.set_xlabel("Survived")
ax.legend(labels=["Title", "Survived"]);

train_encoded["Title-encoded"] = title
train_encoded.info()


In [ ]:
X_enc = train_encoded.loc[:,['Fare_age_group','Title-encoded']]
y_enc = train_encoded['Survived']
make_MI(X_enc, y_enc, discrete_features = False )